# BLAES Units Rhythmicity Analyses

This notebook contains code for analyzing whether spiking rhythmicity is altered by stimulation, as measured by the peaks in the spike timing autocorrelograms.

---

> *Contact: Justin Campbell (justin.campbell@hsc.utah.edu)*  
> *Version: 05/28/2025*

## 1. Import Libraries

In [ ]:
# Libraries
import os
import re
import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy.stats import mannwhitneyu
# from spiketools.measures.conversions import convert_times_to_train
import matplotlib.pyplot as plt
import seaborn as sns

# Notebook settings
%matplotlib inline
%config InlineBackend.figure_format='retina'

## 2. Set Paths & Parameters

In [ ]:
# Params
fs = 30000
export = True

# Define paths
proj_path = '/Users/justincampbell/Library/CloudStorage/GoogleDrive-u0815766@gcloud.utah.edu/My Drive/Research Projects/BLAESUnits/'
results_path = os.path.join(proj_path, 'Results')

# Get list of sessions
sessions = os.listdir(results_path)
sessions = [pID for pID in sessions if re.search(r'\d+$', pID)]

## 3. Spike Timing Autocorrelation

In [ ]:
def ACG(pID, events, stim_epochs, unit, export = False):
    '''
    This function computes the spike-time autocorrelogram (ACG) for a given unit across trials in the stim condition.
    It separates the trials into pre-, during-, and post-stim epochs, computes the ACG for each epoch, 
    normalizes the ACG values by the trialwise spike counts and (optionally) plots the results.
    
    Inputs:
    - pID: string, participant ID
    - events: DataFrame, spike events data
    - stim_epochs: DataFrame, stim-condition epoch indices
    - unit: string, unit identifier in the format 'Channel-Unit' (e.g., 'mLHIP1-1')
    - export: boolean, whether to export the plot (default is False)

    Outputs:
    - acg_df: DataFrame, containing the ACG peak times and values for each epoch

    '''
    
    def autocorrelogram(spike_times, bin_size=0.005, max_lag=0.25):
        '''
        Compute raw spike-time autocorrelogram.

        Parameters
        ----------
        spike_times : 1D array of spike times (in seconds)
        bin_size    : width of each lag bin (seconds)
        max_lag     : maximum lag to consider (seconds)

        Returns
        -------
        counts      : counts in each bin
        bin_edges   : edges of the lag bins
        '''
        
        # compute all pairwise differences
        diffs = spike_times[:, None] - spike_times[None, :]
        # flatten and remove zero‐lag (i.e. self‐pairs)
        diffs = diffs.flatten()
        diffs = diffs[diffs != 0]

        # define lag bins
        bins = np.arange(-max_lag, max_lag + bin_size, bin_size)
        counts, bin_edges = np.histogram(diffs, bins=bins)
        return counts, bin_edges

    ##### COMPUTE ACG
        
    # Get unit data
    unitDF = events.copy()
    unitDF['Chan-Unit'] = unitDF['Channel'].astype(str) + '-' + unitDF['Unit'].astype(str)
    unitDF = unitDF[unitDF['Chan-Unit'] == unit]
    unitDF['TimeStamps'] = unitDF['TimeStamps'].astype('int')
    n_trials = stim_epochs.shape[1]
    epochs = stim_epochs.copy()
    
    # Get spike events and ACGs for each trial
    epochSpikes = []
    counts_list = []
    for i in range(n_trials):
        epoch_start = epochs.iloc[:,i].min()
        stim_start = epoch_start + (4 * fs)
        epoch_end = epochs.iloc[:,i].max()
        epochDF = unitDF[(unitDF['TimeStamps'] >= epoch_start) & (unitDF['TimeStamps'] <= epoch_end)]
        epochDF = epochDF.reset_index(drop = True)
        epochDF['TimeAdj'] = (epochDF['TimeStamps'] - stim_start) / fs
        epochDF['Trial'] = i
        epochDF['Condition'] = 'Stim'
        epochSpikes.append(epochDF)
        
        # Separate the pre-, during-, and post-stim periods
        preDF = epochDF[epochDF['TimeAdj'] < -3]
        duringDF = epochDF[(epochDF['TimeAdj'] >= 0) & (epochDF['TimeAdj'] <= 1)]
        postDF = epochDF[epochDF['TimeAdj'] > 1]
        
        # Compute ACGs
        pre_counts, edges = autocorrelogram(preDF['TimeAdj'].values, bin_size=0.005, max_lag=0.25)
        during_counts, edges = autocorrelogram(duringDF['TimeAdj'].values, bin_size=0.005, max_lag=0.25)
        post_counts, edges = autocorrelogram(postDF['TimeAdj'].values, bin_size=0.005, max_lag=0.25)
        
        # Normalize counts within trial (if there are spikes)
        if np.sum(pre_counts) > 0:
            pre_counts = pre_counts / np.sum(pre_counts)
        if np.sum(during_counts) > 0:
            during_counts = during_counts / np.sum(during_counts)
        if np.sum(post_counts) > 0:
            post_counts = post_counts / np.sum(post_counts)
        counts_list.append((pre_counts, during_counts, post_counts))
        
    # Average across trials
    counts_list = np.mean(counts_list, axis=0)
    
    ##### PLOTTING
    
    # Parameters
    bin_centers = (edges[:-1] + edges[1:]) / 2
    t_fill = np.arange(-0.25, 0.25, 0.005)
    palette = ['#e4e4e4', '#5e4b8b', '#6d9dc5']
    
    # Plotting
    fig, ax = plt.subplots(1, 1, figsize = (3,1.5))
    
    peak_times = []
    peak_values = []
    max_smoothed = np.max([np.max(gaussian_filter1d(counts_list[ii], sigma=1.5)) for ii in range(3)])
    for ii in range(3):

        # Plot line for smoothed ACG
        smoothed_counts = gaussian_filter1d(counts_list[ii], sigma=1.5)
        ax.plot(bin_centers, smoothed_counts, color=palette[ii], linewidth=1.5, label='Smoothed ACG')
        ax.fill_between(t_fill, smoothed_counts, color=palette[ii], alpha=0.2, label='Shaded Area')
        
        # Get peak value and latency, plot both
        greater_than_zero = bin_centers > 0
        if np.any(greater_than_zero):
            peak_index = np.argmax(smoothed_counts[greater_than_zero])
            peak_time = bin_centers[greater_than_zero][peak_index]
            peak_value = smoothed_counts[greater_than_zero][peak_index]
            peak_times.append(peak_time)
            peak_values.append(peak_value)
            ax.scatter(peak_time, peak_value, color=palette[ii], edgecolor = 'k', lw = 1, s=25, label='Peak ACG', zorder = 10)
            ax.plot([peak_time, peak_time], [0, peak_value], color = 'k', linestyle = '--', linewidth=1, label='Peak Lag')
            
    # Aesthetics
    sns.despine(top=True, right=True)
    ax.set_xlabel('Lag (ms)', fontsize='large', labelpad=10)
    ax.set_ylabel('ACG (norm.)', fontsize='large', labelpad=10)
    ax.set_xticks(np.arange(-.25, .26, .125), [-250, -125, 0, 125, 250], fontsize='medium')
    ax.set_xlim(-.25, .25)
    ax.set_ylim(0, max_smoothed * 1.2)
    ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    handles = [
        plt.Line2D([0], [0], color=palette[0], lw=1.5),
        plt.Line2D([0], [0], color=palette[1], lw=1.5),
        plt.Line2D([0], [0], color=palette[2], lw=1.5)]
    labels = ['Pre-', 'During-', 'Post-']
    ax.legend(handles, labels, title='Epoch', fontsize='small', title_fontsize='small', loc='upper right', bbox_to_anchor=(1.45, 1))
    
    if export:
        # Export & Display
        fig_id = 'ACG_' + pID + '_' + unit
        plt.savefig('/Users/justincampbell/Library/CloudStorage/GoogleDrive-u0815766@gcloud.utah.edu/My Drive/Research Projects/BLAESUnits/Results/Group/Figures/ACGs' + '/' + fig_id + '.pdf', dpi=1000, bbox_inches='tight')
        plt.close()
        
    # Construct DF
    acg_df = pd.DataFrame({
        'pID': pID,
        'Unit': unit,
        'Condtion': 'Stim',
        'Pre_Time': peak_times[0],
        'Pre_Peak': peak_values[0],
        'During_Time': peak_times[1],
        'During_Peak': peak_values[1],
        'Post_Time': peak_times[2],
        'Post_Peak': peak_values[2]
    }, index=[0])
    
    return acg_df

In [ ]:
spike_stats = pd.read_csv(os.path.join(results_path, 'Group', 'SpikeStats.csv'), index_col=0)

# Get subset of neurons for ACG analysis (mean FR ≥ 1 Hz across entire trial)
spike_stats = spike_stats[spike_stats['Valid'] == True]
acg_neurons = spike_stats[(spike_stats['FR_Stim_ISI'] >= 1) & 
                        (spike_stats['FR_Img'] >= 1) & 
                        (spike_stats['FR_Stim_During'] >= 1) & 
                        (spike_stats['FR_Stim_Post'] >= 1)]

acg_neurons = acg_neurons.reset_index(drop=False)
acg_neurons

In [ ]:
acg_dfs = []

for i in range(len(acg_neurons)):
    pID = acg_neurons.loc[i, 'pID']
    unit = acg_neurons.loc[i, 'Unit']
    events = pd.read_csv(os.path.join(results_path, pID, 'Events.csv'), index_col=0)
    stim_epochs = pd.read_csv(os.path.join(results_path, pID, 'StimEpochs.csv'), index_col=0)
    df = ACG(pID, events, stim_epochs, unit, export = True)
    acg_dfs.append(df)

In [ ]:
# Concatenate all ACG DataFrames
acg_df = pd.concat(acg_dfs, ignore_index=True)
acg_df = acg_df.reset_index(drop=True)

# Add 'StimSig' column from spike_stats
acg_df = acg_df.merge(spike_stats[['pID', 'Unit', 'StimSig']], on=['pID', 'Unit'], how='left')

# Compute differences between pre- x during- and pre- x post-stim peak times
acg_df['Pre_During_Diff'] = acg_df['Pre_Time'] - acg_df['During_Time']
acg_df['Pre_Post_Diff'] = acg_df['Pre_Time'] - acg_df['Post_Time']

acg_df.to_csv(os.path.join(results_path, 'Group', 'ACGStats.csv'))

## 4. Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(1.25, 1.5))
palette = ['#e4e4e4', '#5e4b8b', '#6d9dc5']

sns.boxplot(data=acg_df, x='StimSig', y='Pre_During_Diff', ax=ax, hue = 'StimSig', palette=[palette[0], palette[1]], fliersize = 2, width = 0.65, linewidth = 1, linecolor = 'k', legend = False)

# Aesthetics
sns.despine(top=True, right=True)
ax.set_xlabel('Pre- vs. During-', fontsize='medium', labelpad=10)
ax.set_ylabel('Peak ACG Diff (ms)', fontsize='medium', labelpad=10)
ax.set_xticks([0, 1], ['NS', 'Mod'], fontsize='medium')
ax.set_yticks(np.arange(-.2, .21, .1), ['-200', '-100', '0', '100', '200'], fontsize='medium')
ax.set_ylim(-.2, .2)

plt.savefig((os.path.join(results_path, 'Group', 'Figures', 'ACGDiff_PreDuring.pdf')), dpi = 1200, bbox_inches = 'tight')
plt.show()

# Pre- vs. During
x1 = acg_df[acg_df['StimSig'] ==  False]['Pre_During_Diff']
x2 = acg_df[acg_df['StimSig'] == True]['Pre_During_Diff']
stat, p_value = mannwhitneyu(x1, x2, alternative='two-sided')
print(f'Mann-Whitney U test Pre- vs. During: U = {stat}, p = {p_value}')

In [ ]:
fig, ax = plt.subplots(figsize=(1.25, 1.5))
palette = ['#e4e4e4', '#5e4b8b', '#6d9dc5']

sns.boxplot(data=acg_df, x='StimSig', y='Pre_Post_Diff', ax=ax, hue = 'StimSig', palette=[palette[0], palette[2]], fliersize = 2, width = 0.65, linewidth = 1, linecolor = 'k', legend = False)

# Aesthetics
sns.despine(top=True, right=True)
ax.set_xlabel('Pre- vs. Post-', fontsize='medium', labelpad=9)
ax.set_ylabel('Peak ACG Diff (ms)', fontsize='medium', labelpad=10)
ax.set_xticks([0, 1], ['NS', 'Mod'], fontsize='medium')
ax.set_yticks(np.arange(-.2, 0.21, .1), ['', '', '', '', ''])
ax.set_ylim(-.2, .2)

plt.savefig((os.path.join(results_path, 'Group', 'Figures', 'ACGDiff_PrePost.pdf')), dpi = 1200, bbox_inches = 'tight')
plt.show()

# Pre- vs. Post
x1 = acg_df[acg_df['StimSig'] == False]['Pre_Post_Diff']
x2 = acg_df[acg_df['StimSig'] == True]['Pre_Post_Diff']
stat, p_value = mannwhitneyu(x1, x2, alternative='two-sided')
print(f'Mann-Whitney U test Pre- vs. During: U = {stat}, p = {p_value}')

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 1, figsize = (3,1.5))

sns.kdeplot(data=acg_df, x='Pre_Time', fill = True, color = palette[0], clip = [0, 0.25])
sns.kdeplot(data=acg_df, x='During_Time', fill = True, color = palette[1], clip = [0, 0.25])
sns.kdeplot(data=acg_df, x='Post_Time', fill = True, color = palette[2], clip = [0, 0.25])

sns.despine(top=True, right=True)
ax.set_xlabel('Peak ACG Lag (ms)', fontsize='large', labelpad=10)
ax.set_ylabel('Density', fontsize='large', labelpad=10)
ax.set_xticks(np.arange(-.25, .26, .125), [-250, -125, 0, 125, 250], fontsize='medium')
ax.set_yticks(np.arange(0, 13, 4), ['0', '4', '8', '12'], fontsize='medium')
ax.set_xlim(0, .25)

plt.savefig((os.path.join(results_path, 'Group', 'Figures', 'ACGLags.pdf')), dpi = 1200, bbox_inches = 'tight')
plt.show()